<a href="https://colab.research.google.com/github/fkaimphilat/compling_hse/blob/main/hw9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 9. Предобученные трансформеры 

### Задание 1 (8 баллов).




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install sklearn pandas matplotlib transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# стандартные библиотеки
import os, re
import numpy as np
from string import punctuation
from time import time
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import Counter
from string import punctuation
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.metrics import classification_report
from sklearn.svm import SVC


# tf и huggingface 
import tensorflow as tf
from transformers import TFAutoModel
from transformers import AutoTokenizer

In [4]:
data = pd.read_csv('/content/drive/My Drive/lenta_sample (1).csv')
data.dropna(subset=['topic', 'text'], inplace=True)

In [18]:
model_bert = TFAutoModel.from_pretrained('bert-base-multilingual-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased')

Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [19]:
X = []

for text in data.title:
    ids = tokenizer.encode(text)
    
    X.append(ids[:512])

X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post', maxlen=512)

id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

y = tf.keras.utils.to_categorical([label2id[label] for label in data.topic.values])

train_index, valid_index = train_test_split(list(range(len(X))), test_size=0.05, stratify=data.topic)

In [20]:
X_train, y_train = X[train_index], y[train_index]
X_valid, y_valid = X[valid_index], y[valid_index]

In [21]:
input_word_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                     name="input_ids")

output = model_bert({"input_ids":input_word_ids})

# добавим дропаут чтобы не переобучалось
drop = tf.keras.layers.Dropout(0.1)(output[0][:, 0]) # [0][:, 0] - означает что мы берем первое 
                                                     # состояние у всех текстов в батче
dense = tf.keras.layers.Dense(y.shape[1], activation='softmax')(drop)

model_clf = tf.keras.Model(inputs=input_word_ids, outputs=dense)

# обычно при дообучении трансформеров нужно ставить очень маленький лосс
# но когда мы обучаем только итоговый классификатор то можно ставить побольше
model_clf.compile(tf.optimizers.Adam(learning_rate=2e-3), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           tf.keras.metrics.RecallAtPrecision(0.80, name='rec_prec')])


In [22]:
model_clf.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=3,
         epochs=10)

Epoch 1/10
192/192 [==============================] - 90s 419ms/step - loss: 3.6293 - accuracy: 0.0938 - rec_prec: 0.0000e+00 - val_loss: 3.1826 - val_accuracy: 0.0645 - val_rec_prec: 0.0000e+00
Epoch 2/10
192/192 [==============================] - 81s 423ms/step - loss: 3.0969 - accuracy: 0.0799 - rec_prec: 0.0000e+00 - val_loss: 3.0049 - val_accuracy: 0.0645 - val_rec_prec: 0.0000e+00
Epoch 3/10
192/192 [==============================] - 81s 424ms/step - loss: 2.9703 - accuracy: 0.0747 - rec_prec: 0.0000e+00 - val_loss: 2.8407 - val_accuracy: 0.0645 - val_rec_prec: 0.0000e+00
Epoch 4/10
192/192 [==============================] - 81s 423ms/step - loss: 2.8548 - accuracy: 0.0608 - rec_prec: 0.0000e+00 - val_loss: 2.7626 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 5/10
192/192 [==============================] - 81s 423ms/step - loss: 2.8398 - accuracy: 0.0642 - rec_prec: 0.0000e+00 - val_loss: 2.7706 - val_accuracy: 0.0645 - val_rec_prec: 0.0000e+00
Epoch 6/10
192/192 [=====

In [24]:
pred = model_clf.predict(X_valid, batch_size=10).argmax(1)

In [25]:
print(classification_report(y_valid.argmax(1), pred, labels=list(range(len(label2id))),
                            target_names=list(label2id), zero_division=0))

                   precision    recall  f1-score   support

       Библиотека       0.00      0.00      0.00         0
   69-я параллель       0.00      0.00      0.00         1
           Россия       0.00      0.00      0.00         2
              Дом       0.00      0.00      0.00         2
          Легпром       0.00      0.00      0.00         1
        Экономика       0.00      0.00      0.00         2
         Ценности       0.00      0.00      0.00         2
   Интернет и СМИ       0.00      0.00      0.00         2
            Спорт       0.00      0.00      0.00         2
  Наука и техника       0.10      1.00      0.18         3
      Бывший СССР       0.00      0.00      0.00         3
         Из жизни       0.00      0.00      0.00         3
Силовые структуры       0.00      0.00      0.00         2
             Крым       0.00      0.00      0.00         0
           Бизнес       0.00      0.00      0.00         2
         Культура       0.00      0.00      0.00       

## Geotrend/bert-base-ru-cased

In [6]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-ru-cased")

model_gt = AutoModelForMaskedLM.from_pretrained("Geotrend/bert-base-ru-cased")

In [7]:
X = []

for text in data.title:
    ids = tokenizer.encode(text)
    
    X.append(ids[:512])
    
X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post', maxlen=512)

id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

y = tf.keras.utils.to_categorical([label2id[label] for label in data.topic.values])

train_index, valid_index = train_test_split(list(range(len(X))), test_size=0.05, stratify=data.topic)

X_train, y_train = X[train_index], y[train_index]
X_valid, y_valid = X[valid_index], y[valid_index]

In [9]:
model_gt = TFAutoModel.from_pretrained("Geotrend/bert-base-ru-cased", trainable=False)

Some layers from the model checkpoint at Geotrend/bert-base-ru-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at Geotrend/bert-base-ru-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [10]:
input_word_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                     name="input_ids")

output = model_gt({"input_ids":input_word_ids})
drop = tf.keras.layers.Dropout(0.3)(output[0][:, 0]) 
dense = tf.keras.layers.Dense(y.shape[1], activation='softmax')(drop)

model_clf = tf.keras.Model(inputs=input_word_ids, outputs=dense)


model_clf.compile(tf.optimizers.Adam(learning_rate=2e-6,
                                    ), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           tf.keras.metrics.RecallAtPrecision(0.80, name='rec_prec')])

In [12]:
model_clf.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=3,
         epochs=10)

Epoch 1/10
192/192 [==============================] - 43s 167ms/step - loss: 3.0608 - accuracy: 0.0625 - rec_prec: 0.0017 - val_loss: 2.9294 - val_accuracy: 0.0323 - val_rec_prec: 0.0000e+00
Epoch 2/10
192/192 [==============================] - 27s 141ms/step - loss: 3.0494 - accuracy: 0.0399 - rec_prec: 0.0000e+00 - val_loss: 2.9224 - val_accuracy: 0.0323 - val_rec_prec: 0.0000e+00
Epoch 3/10
192/192 [==============================] - 27s 142ms/step - loss: 3.0530 - accuracy: 0.0556 - rec_prec: 0.0000e+00 - val_loss: 2.9154 - val_accuracy: 0.0323 - val_rec_prec: 0.0000e+00
Epoch 4/10
192/192 [==============================] - 28s 144ms/step - loss: 2.9892 - accuracy: 0.0590 - rec_prec: 0.0000e+00 - val_loss: 2.9090 - val_accuracy: 0.0323 - val_rec_prec: 0.0000e+00
Epoch 5/10
192/192 [==============================] - 28s 145ms/step - loss: 3.0526 - accuracy: 0.0538 - rec_prec: 0.0000e+00 - val_loss: 2.9020 - val_accuracy: 0.0323 - val_rec_prec: 0.0000e+00
Epoch 6/10
192/192 [=========

In [13]:
pred = model_clf.predict(X_valid, batch_size=5).argmax(1)

In [14]:
print(classification_report(y_valid.argmax(1), pred, labels=list(range(len(label2id))),
                            target_names=list(label2id), zero_division=0))

                   precision    recall  f1-score   support

Силовые структуры       0.00      0.00      0.00         2
  Наука и техника       0.00      0.00      0.00         3
   Интернет и СМИ       0.00      0.00      0.00         2
         Ценности       0.00      0.00      0.00         2
        Экономика       0.00      0.00      0.00         2
           Бизнес       0.00      0.00      0.00         2
              Мир       0.00      0.00      0.00         1
         Культура       0.22      0.67      0.33         3
             Крым       0.00      0.00      0.00         0
      Бывший СССР       0.00      0.00      0.00         3
         Из жизни       0.00      0.00      0.00         3
   69-я параллель       0.05      1.00      0.10         1
              Дом       0.00      0.00      0.00         2
           Россия       0.00      0.00      0.00         2
          Легпром       0.00      0.00      0.00         1
       Библиотека       0.00      0.00      0.00       

## Skoltech/russian-sensitive-topics



In [22]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('Skoltech/russian-sensitive-topics')

model_sk = TFAutoModel.from_pretrained('Skoltech/russian-sensitive-topics')

Downloading:   0%|          | 0.00/524 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/680M [00:00<?, ?B/s]

Some layers from the model checkpoint at Skoltech/russian-sensitive-topics were not used when initializing TFBertModel: ['classifier', 'dropout_37']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at Skoltech/russian-sensitive-topics.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [23]:
X = []

for text in data.title:
    ids = tokenizer.encode(text)
    
    X.append(ids[:512])
    
X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post', maxlen=512)

id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

y = tf.keras.utils.to_categorical([label2id[label] for label in data.topic.values])

train_index, valid_index = train_test_split(list(range(len(X))), test_size=0.05, stratify=data.topic)

X_train, y_train = X[train_index], y[train_index]
X_valid, y_valid = X[valid_index], y[valid_index]

In [24]:
model_sk = TFAutoModel.from_pretrained('Skoltech/russian-sensitive-topics', trainable=False)

Some layers from the model checkpoint at Skoltech/russian-sensitive-topics were not used when initializing TFBertModel: ['classifier', 'dropout_37']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at Skoltech/russian-sensitive-topics.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [25]:
input_word_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                     name="input_ids")

output = model_sk({"input_ids":input_word_ids})
drop = tf.keras.layers.Dropout(0.3)(output[0][:, 0]) 
dense = tf.keras.layers.Dense(y.shape[1], activation='softmax')(drop)

model_clf = tf.keras.Model(inputs=input_word_ids, outputs=dense)


model_clf.compile(tf.optimizers.Adam(learning_rate=2e-6,
                                    ), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           tf.keras.metrics.RecallAtPrecision(0.80, name='rec_prec')])

In [26]:
model_clf.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=3,
         epochs=10)

Epoch 1/10
192/192 [==============================] - 38s 164ms/step - loss: 3.4199 - accuracy: 0.0434 - rec_prec: 0.0000e+00 - val_loss: 3.1804 - val_accuracy: 0.0323 - val_rec_prec: 0.0000e+00
Epoch 2/10
192/192 [==============================] - 28s 148ms/step - loss: 3.3617 - accuracy: 0.0660 - rec_prec: 0.0017 - val_loss: 3.1457 - val_accuracy: 0.0323 - val_rec_prec: 0.0000e+00
Epoch 3/10
192/192 [==============================] - 28s 148ms/step - loss: 3.3778 - accuracy: 0.0469 - rec_prec: 0.0000e+00 - val_loss: 3.1153 - val_accuracy: 0.0323 - val_rec_prec: 0.0000e+00
Epoch 4/10
192/192 [==============================] - 28s 148ms/step - loss: 3.3976 - accuracy: 0.0417 - rec_prec: 0.0000e+00 - val_loss: 3.0872 - val_accuracy: 0.0323 - val_rec_prec: 0.0000e+00
Epoch 5/10
192/192 [==============================] - 28s 148ms/step - loss: 3.3200 - accuracy: 0.0625 - rec_prec: 0.0000e+00 - val_loss: 3.0613 - val_accuracy: 0.0323 - val_rec_prec: 0.0000e+00
Epoch 6/10
192/192 [=========

In [27]:
pred = model_clf.predict(X_valid, batch_size=5).argmax(1)

In [28]:
print(classification_report(y_valid.argmax(1), pred, labels=list(range(len(label2id))),
                            target_names=list(label2id), zero_division=0))

                   precision    recall  f1-score   support

Силовые структуры       0.00      0.00      0.00         2
  Наука и техника       0.00      0.00      0.00         3
   Интернет и СМИ       0.00      0.00      0.00         2
         Ценности       0.00      0.00      0.00         2
        Экономика       0.00      0.00      0.00         2
           Бизнес       0.00      0.00      0.00         2
              Мир       0.00      0.00      0.00         1
         Культура       0.33      1.00      0.50         3
             Крым       0.00      0.00      0.00         0
      Бывший СССР       0.00      0.00      0.00         3
         Из жизни       0.00      0.00      0.00         3
   69-я параллель       0.00      0.00      0.00         1
              Дом       0.00      0.00      0.00         2
           Россия       0.00      0.00      0.00         2
          Легпром       0.06      1.00      0.11         1
       Библиотека       0.00      0.00      0.00       

## bert-base-multilingual-cased

In [37]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

model_bbms = TFAutoModel.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [38]:
X = []

for text in data.title:
    ids = tokenizer.encode(text)
    
    X.append(ids[:512])
    
X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post', maxlen=512)

id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

y = tf.keras.utils.to_categorical([label2id[label] for label in data.topic.values])

train_index, valid_index = train_test_split(list(range(len(X))), test_size=0.05, stratify=data.topic)

X_train, y_train = X[train_index], y[train_index]
X_valid, y_valid = X[valid_index], y[valid_index]

In [39]:
model_bbms = TFAutoModel.from_pretrained('bert-base-multilingual-cased', trainable=False)

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [40]:
input_word_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                     name="input_ids")

output = model_bbms({"input_ids":input_word_ids})
drop = tf.keras.layers.Dropout(0.3)(output[0][:, 0]) 
dense = tf.keras.layers.Dense(y.shape[1], activation='softmax')(drop)

model_clf = tf.keras.Model(inputs=input_word_ids, outputs=dense)


model_clf.compile(tf.optimizers.Adam(learning_rate=2e-6,
                                    ), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           tf.keras.metrics.RecallAtPrecision(0.80, name='rec_prec')])

In [41]:
model_clf.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=3,
         epochs=10)

Epoch 1/10
192/192 [==============================] - 40s 162ms/step - loss: 3.0617 - accuracy: 0.0556 - rec_prec: 0.0017 - val_loss: 2.9781 - val_accuracy: 0.1290 - val_rec_prec: 0.0000e+00
Epoch 2/10
192/192 [==============================] - 29s 150ms/step - loss: 3.1382 - accuracy: 0.0556 - rec_prec: 0.0000e+00 - val_loss: 2.9690 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 3/10
192/192 [==============================] - 28s 147ms/step - loss: 3.0786 - accuracy: 0.0625 - rec_prec: 0.0000e+00 - val_loss: 2.9602 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 4/10
192/192 [==============================] - 28s 147ms/step - loss: 3.1206 - accuracy: 0.0417 - rec_prec: 0.0000e+00 - val_loss: 2.9518 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 5/10
192/192 [==============================] - 29s 150ms/step - loss: 3.1126 - accuracy: 0.0451 - rec_prec: 0.0000e+00 - val_loss: 2.9436 - val_accuracy: 0.0968 - val_rec_prec: 0.0000e+00
Epoch 6/10
192/192 [=========

In [44]:
pred = model_clf.predict(X_valid, batch_size=5).argmax(1)

In [45]:
print(classification_report(y_valid.argmax(1), pred, labels=list(range(len(label2id))),
                            target_names=list(label2id), zero_division=0))

                   precision    recall  f1-score   support

Силовые структуры       0.00      0.00      0.00         2
  Наука и техника       1.00      0.33      0.50         3
   Интернет и СМИ       0.00      0.00      0.00         2
         Ценности       0.00      0.00      0.00         2
        Экономика       0.00      0.00      0.00         2
           Бизнес       0.00      0.00      0.00         2
              Мир       0.00      0.00      0.00         1
         Культура       0.00      0.00      0.00         3
             Крым       0.00      0.00      0.00         0
      Бывший СССР       0.00      0.00      0.00         3
         Из жизни       0.00      0.00      0.00         3
   69-я параллель       0.00      0.00      0.00         1
              Дом       0.00      0.00      0.00         2
           Россия       0.07      1.00      0.13         2
          Легпром       0.00      0.00      0.00         1
       Библиотека       0.00      0.00      0.00       

Вообще все модели получились слабыми - я даже ради интереса попробовал SVM, и получил лучший результат.

Однако среди представленных моделей лучше всез себя показала  **Skoltech/russian-sensitive-topics**.

### Задание  2 (2 балла)


1) Чем ROBERTA отличается от BERT? В какой статье описана ROBERTA?

2) Чем модель T5 отличается от ROBERTA/BERT? 

1) Проанализировав работу гугловской модели BERT, команда из Facebook AI и Университета Вашингтона нашла способы её улучшить, в частности: они использовали 160 гигабайт текстов вместо 16 в BERT; увеличили кол-во итераций с 100000 до 300000, а затем и до 500000; к обучающим данным было применено динамическое изменение шаблона маскировки; и была исключена задача предсказания следующей последовательности из процедуры обучения. По итогу, ROBERTA показывает себя лучше, чем BERT на бенчмарках GLUE.

ссылка 1: https://towardsdatascience.com/exploring-bert-variants-albert-roberta-electra-642dfe51bc23

ссылка 2: https://iq.opengenus.org/advanced-nlp-models/


2) Одна из самых продвинутых NLP моделей от Google, T5 построена на основе предыдущих работ с трансформерами, и в отличие от BERT, в которой используются только блоки энкодера, в T5 используются также и блоки декодеры. К числу изменений по сравнению с BERT можно отнести: добавление причинного декодера к двунаправленной архитектуре, замену задачи "fill-in-the-blank cloze" на сочетание альтернативных задач предварительного обучения.

ссылка 1: https://arxiv.org/abs/1910.10683v3
